In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

 # linear algebra


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

    
        

In [ ]:
import numpy as ny
import pandas as ps
import plotly.express as plex

# Modelling
from sklearn import  linear_model
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

amsterdam = ps.read_csv("../input/amsterdam-house-price-prediction/HousingPrices-Amsterdam-August-2021.csv")
amsterdam.head(10)



------------------------------
Initial thoughts:

Purpose: To get a sensible price prediction in number format.

1. Addresses are long strings, may be we could get sub-locality as an input( Keeping this for 2nd training iteration)
2. Zip has to be cleaned for number format ?
3. Check for outliers in Area, Price and rooms.
4. For now let's drop the Lat/Lon as zip/address gives a close swing.
------------------------------

# Step 1: Data cleanup.

In [ ]:
#Saving addresses.

addresses = amsterdam['Address']

prices = amsterdam['Price']
clean1 = amsterdam.drop(columns=['Unnamed: 0','Address'])

print("\n Shape is: ",clean1.shape)
clean1.head(10)



In [ ]:
clean1.describe() #info() # Data looks clean and outliers don't seem to be persistent

In [ ]:
#checking null values
clean1.isnull().sum() 


In [ ]:
clean1 = clean1.dropna() 
print("\n Shape : ",clean1.shape) # Not many rows lost. Great!

In [ ]:
# Transforming ZIP code to string and then slicing to use 1st part as a int. Let's see if that helps.


clean1['Zip'] = clean1['Zip'].str.split(' ',1).str[0].astype(ny.int32)

clean1.head(10) #.info() -> confirm datatype change.


# Step 2: Visualizing the numeric dataframe now.

In [ ]:
# Which zip codes lead with prices

barplot1 = plex.bar(clean1, x='Zip',y='Price',title="Zip code vs Price - Alongside room count",color='Room')
barplot1.show()

In [ ]:
heatmap = plex.imshow(clean1.corr(),title="Co-realtion Heatmap")
heatmap.show()

In [ ]:
# At this point, converting to zip shows less corr towards price.

# Step 3: Modelling

In [ ]:
# Defining X and y
y = clean1['Price']
X = clean1.drop(columns = ['Price'])

# T-T-S
X_train,X_test,y_train,y_test = train_test_split(X, y, train_size=0.76,random_state=42)

# Linear reg. model
lr_model_without_zipcode = linear_model.LinearRegression()
lr_model_without_zipcode.fit(X_train, y_train)

y_predicted = lr_model_without_zipcode.predict(X_test)

print("\n Train set score: ",lr_model_without_zipcode.score(X_train,y_train))
print("\n R-Squared : ", r2_score(y_test, y_predicted) * 100)

In [ ]:
## Same model with ZIP CODE

y = clean1['Price']
X = clean1.drop(columns = ['Price','Zip'])

# T-T-S
X_train,X_test,y_train,y_test = train_test_split(X, y, train_size=0.75,random_state=42)

# Linear reg. model
lr_model_with_zipcode = linear_model.LinearRegression()
lr_model_with_zipcode.fit(X_train, y_train)

y_predicted = lr_model_with_zipcode.predict(X_test)

print("\n Train set score ( With Zip Code): ",lr_model_with_zipcode.score(X_train,y_train))
print("\n R-Squared ( With Zip Code) : ", r2_score(y_test, y_predicted) * 100)

In [ ]:
# Let me know your thoughts on this one !!!